# Lesson 2 – Solutions Notebook  
Multiple Inputs Pattern: Daily Expense Summary Agent

This notebook contains **one possible solution** for the exercises in:

`02_multiple_inputs_daily_expense_summary.ipynb`

Your own solutions may differ but still be valid as long as they achieve the same behavior.


## 1. Imports and State Definitions

We extend the original `ExpenseState` to include:

- `target_currency: str` – currency to convert into (Exercise 2).
- `converted_total: float` – total spent converted into `target_currency` (Exercise 2).
- `alert: bool` – flag when the user is significantly over budget (Stretch Exercise 2).
- A more detailed `Expense` structure with categories (Stretch Exercise 1).


In [ ]:
from typing import TypedDict, List, Optional
from collections import defaultdict

class Expense(TypedDict):
    amount: float
    category: str  # e.g. "food", "transport", "other"


class ExpenseState(TypedDict):
    expenses: List[Expense]
    currency: str
    daily_budget: float
    total_spent: float
    num_transactions: int
    summary: str

    # Extensions
    target_currency: str
    converted_total: float
    alert: bool

## 2. Helper Functions

We define a few small helper functions to keep the main node readable.


In [ ]:
def convert_currency(amount: float, rate: float) -> float:
    """Fake currency conversion helper.

    In a real application, you'd use live FX rates.
    Here we just multiply by a fixed rate for demonstration.
    """
    return amount * rate


def compute_category_totals(expenses: List[Expense]) -> dict:
    """Compute total spend per category."""
    totals = defaultdict(float)
    for e in expenses:
        totals[e["category"]] += e["amount"]
    return totals


## 3. Final `summarize_expenses` Node Implementation

This implementation covers:

1. **Zero-expense days** (Exercise 1).
2. **Multiple currencies** with `target_currency` and `converted_total` (Exercise 2).
3. **Category breakdown** and mention of top category (Stretch Exercise 1).
4. **Alert flag** when user is more than 20% over budget (Stretch Exercise 2).


In [ ]:
def summarize_expenses(state: ExpenseState) -> ExpenseState:
    """Comprehensive summarizer node for Lesson 2 exercises."""
    expenses = state.get("expenses", [])
    currency = state.get("currency", "EUR")
    budget = state.get("daily_budget", 0.0)
    target_currency = state.get("target_currency", currency)

    # 1. Zero-expense day handling
    if not expenses:
        state["total_spent"] = 0.0
        state["num_transactions"] = 0
        state["converted_total"] = 0.0
        state["alert"] = False
        state["summary"] = "No expenses recorded today."
        return state

    # Compute base totals
    total = sum(e["amount"] for e in expenses)
    num = len(expenses)

    # Basic budget comparison
    remaining = budget - total
    if budget > 0:
        if remaining > 0:
            status = f"You are {remaining:.2f} {currency} under budget."
        elif remaining == 0:
            status = "You hit your budget exactly today."
        else:
            status = f"You are {-remaining:.2f} {currency} over budget."
    else:
        status = "No budget set."

    # 2. Currency conversion (fake fixed rate)
    # For demo purposes, we define a simple rate rule:
    # - If target_currency != currency, use rate 1.1
    # - Else rate = 1.0
    rate = 1.1 if target_currency != currency else 1.0
    converted_total = convert_currency(total, rate)

    # 3. Category breakdown
    category_totals = compute_category_totals(expenses)
    if category_totals:
        top_category, top_amount = max(category_totals.items(), key=lambda kv: kv[1])
        category_msg = (
            f" Your top spending category was {top_category} ({top_amount:.2f} {currency})."
        )
    else:
        category_msg = ""

    # 4. Alert flag (20% over budget)
    alert = False
    alert_msg = ""
    if budget > 0 and total > 1.2 * budget:
        alert = True
        alert_msg = " Warning: you are more than 20% over your daily budget!"

    # Build the summary string
    if target_currency != currency:
        summary = (
            f"You spent {total:.2f} {currency} today (~{converted_total:.2f} {target_currency}) "
            f"across {num} transactions. {status}{category_msg}{alert_msg}"
        )
    else:
        summary = (
            f"You spent {total:.2f} {currency} today across {num} transactions. "
            f"{status}{category_msg}{alert_msg}"
        )

    # Write everything back into the state
    state["total_spent"] = total
    state["num_transactions"] = num
    state["summary"] = summary
    state["converted_total"] = converted_total
    state["alert"] = alert
    return state


## 4. Wiring Up the LangGraph App

We now wrap this node into a `StateGraph`, like in the teaching notebook.


In [ ]:
from langgraph.graph import StateGraph

graph = StateGraph(ExpenseState)
graph.add_node("summarize_expenses", summarize_expenses)
graph.set_entry_point("summarize_expenses")
graph.set_finish_point("summarize_expenses")  # single-node graph

app = graph.compile()

## 5. Example Calls

Below are several test cases that demonstrate the implemented behavior.


In [ ]:
# 5.1 Zero-expense day
state_zero: ExpenseState = {
    "expenses": [],
    "currency": "EUR",
    "daily_budget": 50.0,
    "total_spent": 0.0,
    "num_transactions": 0,
    "summary": "",
    "target_currency": "USD",
    "converted_total": 0.0,
    "alert": False,
}
app.invoke(state_zero)

In [ ]:
# 5.2 Normal day under budget
state_under: ExpenseState = {
    "expenses": [
        {"amount": 10.0, "category": "food"},
        {"amount": 5.0, "category": "transport"},
        {"amount": 8.0, "category": "food"},
    ],
    "currency": "EUR",
    "daily_budget": 50.0,
    "total_spent": 0.0,
    "num_transactions": 0,
    "summary": "",
    "target_currency": "USD",
    "converted_total": 0.0,
    "alert": False,
}
app.invoke(state_under)

In [ ]:
# 5.3 Over budget with alert
state_over: ExpenseState = {
    "expenses": [
        {"amount": 40.0, "category": "shopping"},
        {"amount": 30.0, "category": "shopping"},
        {"amount": 5.0, "category": "food"},
    ],
    "currency": "EUR",
    "daily_budget": 50.0,
    "total_spent": 0.0,
    "num_transactions": 0,
    "summary": "",
    "target_currency": "EUR",
    "converted_total": 0.0,
    "alert": False,
}
app.invoke(state_over)

You can modify these test cases or add more to further explore the behavior.

This notebook provides one clean, worked solution for **Lesson 2 – Multiple Inputs Pattern**.  
You can keep it separate from the teaching notebook to avoid spoilers for learners.
